## 1. importar los archivos .csv con los dos tipos de nodos y los arcos en Python, crear la red bipartita con la librería networkx

In [1]:
#Carga librerías importantes
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import operator
import pip
import csv
import pickle
import operator
from networkx.algorithms import bipartite
import random


In [2]:
#Read SECOP network

#Create empty network

SECOP= nx.Graph()

entidades=csv.reader(open('./Nodos_entidad(1).csv',encoding='Latin-1'),delimiter=';')

next(entidades,None)
#read nodes
for row in entidades:
    nodename = 'E-' + row[0]
    SECOP.add_node(nodename,bipartite=0)
    SECOP.node[nodename]['nombre']=row[1]

contratistas=csv.reader(open('./Nodos_contratista(1).csv',encoding='Latin-1'),delimiter=';')
next(contratistas,None)

for row in contratistas:
    nodename = 'C-' + row[0]
    SECOP.add_node(nodename,bipartite=1)
    SECOP.node[nodename]['nombre']=row[1]
    

    
#read edges
edges=csv.reader(open('./Arcos_procesos(1).csv',encoding='Latin-1'),delimiter=';')
                 
next(edges,None)
for row in edges:
    nodename_e = 'E-' + row[0]
    nodename_c = 'C-' + row[1]
    if(nodename_e in SECOP.nodes()) and (nodename_c in SECOP.nodes()):
        SECOP.add_edge(nodename_e,nodename_c)
        SECOP.edge[nodename_e][nodename_c]['Estado del proceso']=row[2]
        SECOP.edge[nodename_e][nodename_c]['Objeto a contratar']=row[3]
        SECOP.edge[nodename_e][nodename_c]['Origen recursos']=row[4]
        SECOP.edge[nodename_e][nodename_c]['Valor contrato con adiciones']=row[5]

print('La red tiene ',SECOP.number_of_nodes(),'nodos y ',SECOP.number_of_edges(),'arcos')

print(nx.is_bipartite(SECOP))


La red tiene  42322 nodos y  44022 arcos
True


### 1.A. Proyección desde la red bipartita a una red de entidades con pesos

In [3]:
entidades = set(nodes for nodes,tipo in SECOP.nodes(data=True) if tipo['bipartite']==0)
entidades_net= bipartite.weighted_projected_graph(SECOP,entidades)
print('La red tiene ',entidades_net.number_of_nodes(),'nodos y ',entidades_net.number_of_edges(),'arcos')

La red tiene  262 nodos y  6401 arcos


### 1.B. Proyección desde la red bipartita a una red de contratistas con pesos.

In [ ]:
contratistas = set(nodes for nodes,tipo in SECOP.nodes(data=True) if tipo['bipartite']==1)
contratistas2=random.sample(contratistas,100)
contratistas_net= bipartite.weighted_projected_graph(SECOP,contratistas)
print('La red tiene ',contratistas_net.number_of_nodes(),'nodos y ',contratistas_net.number_of_edges(),'arcos')

La red tiene  42060 nodos y  22633201 arcos


### 2.A. Calcule la cantidad promedio de conexiones. 
 *Utilizo el grado sin pesos porque para el análisis me parece más interesante saber con cuántas entidades comparto contratistas(caso red de entidades) que saber cuántos contratistas comparto con otras entidades* 

In [5]:
#Entidades
grado=entidades_net.degree()
grado_promedio_simple= sum(grado.values())/entidades_net.number_of_nodes()
print('La cantidad promedio de conexiones en la red de entidades es: ', grado_promedio_simple, 
      '. Es decir, en promedio una entidad tiene contratistas en común con otras',grado_promedio_simple, 'entidades')

#Contratistas
grado2=contratistas_net.degree()
grado2_promedio_simple= sum(grado2.values())/contratistas_net.number_of_nodes()
print('La cantidad promedio de conexiones en la red de contratistas es: ', grado2_promedio_simple,
      '. Es decir, en promedio un contratista tiene entidades en común con otros',grado2_promedio_simple, 'contratistas')

La cantidad promedio de conexiones en la red de entidades es:  48.86259541984733 . Es decir, en promedio una entidad tiene contratistas en común con otras 48.86259541984733 entidades
La cantidad promedio de conexiones en la red de contratistas es:  1076.2339990489777 . Es decir, en promedio un contratista tiene entidades en común con otros 1076.2339990489777 contratistas


### 2.B. La transitividad promedio de la red.

In [6]:
#Entidades
trans_e=nx. average_clustering(entidades_net)
print('La transitividad promedio en la red de entidades es:',trans_e)

#contratistas
trans_c=nx. average_clustering(contratistas_net)
print('La transitividad promedio en la red de contratistas es:',trans_c)


La transitividad promedio en la red de entidades es: 0.5371223061015926


### 2.C. Fracción de conexiones reales sobre conexiones posibles.

In [7]:
#Entidades
densidad_e= nx.density(entidades_net)
print('La proporción de conexiones reales sobre conexiones posibles en la red de entidades es:', densidad_e)

#Contratistas
densidad_c=nx.density(contratistas_net)
print('La proporción de conexiones reales sobre conexiones posibles en la red de contratistas es:', densidad_c)

La proporción de conexiones reales sobre conexiones posibles en la red de entidades es: 0.1872130092714457
La proporción de conexiones reales sobre conexiones posibles en la red de contratistas es: 0.02558867303190703


### 2.D. Longitud promedio del camino entre todo par de nodos

In [ ]:
#Entidades
maximo_conectado_e=max(nx.connected_component_subgraphs(entidades_net),key=len)
shortest_path_length_average_e= nx.average_shortest_path_length(maximo_conectado_e,weight='weight')
print('La cercanía  en la red de entidades es:', shortest_path_length_average_e)
#Contratistas
maximo_conectado_c=max(nx.connected_component_subgraphs(contratistas_net),key=len)
shortest_path_length_average_c= nx.average_shortest_path_length(maximo_conectado_c,weight='weight')
cercania_c=nx.closeness_centrality(contratistas_net)
print('La cercanía  en la red de contatistas es:', shortest_path_length_average_c)

La cercanía  en la red de entidades es: 1.971097068268932


### 3.A. 5 nodos con mayor Grado

In [ ]:
#Entidades

grado_e_ordenado= sorted(entidades_net.degree_iter(weight='weight'),key=operator.itemgetter(1),reverse=True)
print('Las 5 entidades con mayor grado son:', grado_e_ordenado[0:5])

#Contatistas

grado_c_ordenado= sorted(contratistas_net.degree_iter(weight='weight'),key=operator.itemgetter(1),reverse=True)
print('Los 5 contratistas con mayor grado son:', grado_c_ordenado[0:5])


Las 5 entidades con mayor grado son: [('E-899999034', 899), ('E-800140603', 817), ('E-EJERCITO NACIONAL', 539), ('E-800152783', 522), ('E-899999239', 493)]


### 3.B. 5 nodos con mayor Betweenness

In [10]:
#Entidades
betweenness_e= nx.betweenness_centrality(entidades_net,weight='weight')
betweenness_e= [(k,v) for k, v in betweenness_e.items()]
betweenness_e_ordenadas=sorted(betweenness_e,key=operator.itemgetter(1),reverse=True)     
print('Las 5 entidades con mayor Betweenness son:', betweenness_e_ordenadas[0:5])

#Contratistas
betweenness_c= nx.betweenness_centrality(entidades_net,weight='weight')
betweenness_c= [(k,v) for k, v in betweenness_c.items()]
betweenness_c_ordenadas=sorted(betweenness_c,key=operator.itemgetter(1),reverse=True)     
print('Los 5 contratistas con mayor Betweenness son:', betweenness_c_ordenadas[0:5])

Las 5 entidades con mayor Betweenness son: [('E-830125996', 0.014128248936265938), ('E-860011153-6', 0.011984081309424), ('E-EJERCITO NACIONAL', 0.011204024155747741), ('E-800214417-9', 0.010280258045703622), ('E-899999239', 0.00915773900945805)]
Los 5 contratistas con mayor Betweenness son: [('E-830125996', 0.014128248936265938), ('E-860011153-6', 0.011984081309424), ('E-EJERCITO NACIONAL', 0.011204024155747741), ('E-800214417-9', 0.010280258045703622), ('E-899999239', 0.00915773900945805)]


### 3.C. 5 nodos con mayor coeficiente de Clustering 

In [ ]:
#Entidades
clustering_e= nx.clustering(entidades_net,weight='weight')
clustering_e= [(k,v) for k, v in clustering_e.items()]
clustering_e_ordenadas=sorted(clustering_e,key=operator.itemgetter(1),reverse=True)  
print('Las  5 entidades con mayor coeficiente de clustering son:',clustering_e_ordenadas[0:5])
#Contratistas
clustering_c= nx.clustering(contratistas_net,weight='weight')
clustering_c= [(k,v) for k, v in clustering_c.items()]
clustering_c_ordenadas=sorted(clustering_c,key=operator.itemgetter(1),reverse=True)  
print('Las  5 contratistas con mayor coeficiente de clustering son:',clustering_c_ordenadas[0:5])


Las  5 entidades con mayor coeficiente de clustering son: [('E-800072977-0', 0.05421791617343344), ('E-899999034', 0.033511744540787536), ('E-899999044-3', 0.032758846034155714), ('E-DIVISION DE SANIDAD DE LA POLICI\x8dA NACIONAL', 0.031257693900777504), ('E-800140603', 0.03020742550974177)]


### 3.D. 5 nodos con mayor Closeness

In [6]:
#Entidades
closeness_e= nx.closeness_centrality(entidades_net)
closeness_e= [(k,v) for k, v in closeness_e.items()]
closeness_e_ordenadas=sorted(closeness_e,key=operator.itemgetter(1),reverse=True) 
print('Las  5 entidades con mayor closeness son:',closeness_e[0:5])

#Contratistas
closeness_c= nx.closeness_centrality(contratistas_net)
closeness_c= [(k,v) for k, v in closeness_c.items()]
closeness_c_ordenadas=sorted(closeness_c,key=operator.itemgetter(1),reverse=True) 
print('Los 5 contratistas con mayor closeness son:',closeness_c[0:5])

Las  5 entidades con mayor closeness son: [('E-900948953-8', 0.5313636803740127), ('E-900003409-7', 0.4473791473057797), ('E-CORPOAMAZONI\x8dA - CORPORACION PARA EL DESARROLLO SOSTENIBLE SUR DE LA AMAZONI\x8dA', 0.3192792613093309), ('E-900657800-0', 0.4266311288800045), ('E-800248004', 0.24654562724221363)]


### 3.C. 5 nodos con mayor PageRank

In [7]:
#Entidades
pagerank_e=nx.pagerank(entidades_net,weight='weight')
pagerank_e= [(k,v) for k, v in pagerank_e.items()]
pagerank_e_ordenadas=sorted(pagerank_e,key=operator.itemgetter(1),reverse=True) 
print('Las  5 entidades con mayor pagerank son:',pagerank_e[0:5])

#Contratistas
pagerank_c=nx.pagerank(contratistas_net,weight='weight')
pagerank_c= [(k,v) for k, v in pagerank_c.items()]
pagerank_c_ordenadas=sorted(pagerank_c,key=operator.itemgetter(1),reverse=True) 
print('Los 5 contratistas con mayor pagerank son:',pagerank_c[0:5])


Las  5 entidades con mayor pagerank son: [('E-900948953-8', 0.008073327645156055), ('E-900003409-7', 0.0036293452008170192), ('E-CORPOAMAZONI\x8dA - CORPORACION PARA EL DESARROLLO SOSTENIBLE SUR DE LA AMAZONI\x8dA', 0.0007827423235201164), ('E-900657800-0', 0.0025345579215599917), ('E-800248004', 0.0008310675344746685)]


### 4. Haga una matriz de correlación entre las medidas y concluya cuales son las entidades o empresas contratistas con mayor incidencia en la contratación estatal.

In [ ]:
#Entidades
for n in entidades_net.nodes():
    entidades_net.node[n]['grado_ponderado']=grado_e[n]
    entidades_net.node[n]['intermediacion_ponderado']=betweenness_e[n]
    entidades_net.node[n]['clustering_ponderado']=clustering_e[n]
    entidades_net.node[n]['cercania_ponderado']=closeness_e[n]
    entidades_net.node[n]['pagerank_ponderado']=pagerank_e[n]
print('Asortatividad_e grado', nx.attribute_assortativity_coefficient(entidades_net,attribute='grado_ponderado'))
print('Asortatividad_e intermediacion', nx.attribute_assortativity_coefficient(entidades_net,attribute='intermediacion_ponderado')) 
print('Asortatividad_e clustering', nx.attribute_assortativity_coefficient(entidades_net,attribute='clustering_ponderado')) 
print('Asortatividad_e cercania', nx.attribute_assortativity_coefficient(entidades_net,attribute='cercania_ponderado')) 
print('Asortatividad_e pagerank', nx.attribute_assortativity_coefficient(entidades_net,attribute='pagerank_ponderado')) 

#Contratistas
for n in contratistas_net.nodes():
    contratistas_net.node[n]['grado_ponderado']=grado_c[n]
    contratistas_net.node[n]['intermediacion_ponderado']=betweenness_c[n]
    contratistas_net.node[n]['clustering_ponderado']=clustering_c[n]
    contratistas_net.node[n]['cercania_ponderado']=closeness_c[n]
    contratistas_net.node[n]['pagerank_ponderado']=pagerank_c[n]
print('Asortatividad_c grado', nx.attribute_assortativity_coefficient(contratistas_net,attribute='grado_ponderado'))
print('Asortatividad_c intermediacion', nx.attribute_assortativity_coefficient(contratistas_net,attribute='intermediacion_ponderado')) 
print('Asortatividad_c clustering', nx.attribute_assortativity_coefficient(contratistas_net,attribute='clustering_ponderado')) 
print('Asortatividad_c cercania', nx.attribute_assortativity_coefficient(contratistas_net,attribute='cercania_ponderado')) 
print('Asortatividad_c pagerank', nx.attribute_assortativity_coefficient(contratistas_net,attribute='pagerank_ponderado')) 


### 5. Bono: Grafique ambas redes en Gephi de tal manera que pueda inferir información relevante acerca de los procesos de contratación nacionales del año 2017.

In [ ]:
    #Genera visualización
def grafica(grafo):
    figure=plt.figure(figsize=(20,20))
    pos=nx.shell_layout(grafo)
    nx.draw(grafo, pos=pos, node_size= 300, node_color='green',edge_color='blue')
    plt.show()

grafica(entidades_net)